In [18]:
import scipy.misc
import random
import imageio
from skimage.transform import resize
xs = []
ys = []

#points to the end of the last batch
train_batch_pointer = 0
val_batch_pointer = 0

#read data.txt
with open("driving_dataset/data.txt") as f:
    for line in f:
        xs.append("driving_dataset/" + line.split()[0])
        #the paper by Nvidia uses the inverse of the turning radius,
        #but steering wheel angle is proportional to the inverse of turning radius
        #so the steering wheel angle in radians is used as the output
        ys.append(float(line.split()[1]) * scipy.pi / 180)

#get number of images
num_images = len(xs)


train_xs = xs[:int(len(xs) * 0.7)]
train_ys = ys[:int(len(xs) * 0.7)]

val_xs = xs[-int(len(xs) * 0.3):]
val_ys = ys[-int(len(xs) * 0.3):]

num_train_images = len(train_xs)
num_val_images = len(val_xs)

def LoadTrainBatch(batch_size):
    global train_batch_pointer
    x_out = []
    y_out = []
    for i in range(0, batch_size):
        image = full_image[-150:]/255.0
        image=resize(image,[66,200])
        x_out.append(resize(imread(train_xs[(train_batch_pointer + i) % num_train_images])[-150:]/255.0, [66, 200]) )
        y_out.append([train_ys[(train_batch_pointer + i) % num_train_images]])
    train_batch_pointer += batch_size
    return x_out, y_out

def LoadValBatch(batch_size):
    global val_batch_pointer
    x_out = []
    y_out = []
    for i in range(0, batch_size):
        x_out.append(resize(imread(val_xs[(val_batch_pointer + i) % num_val_images])[-150:]/255.0, [66, 200]))
        y_out.append([val_ys[(val_batch_pointer + i) % num_val_images]])
    val_batch_pointer += batch_size
    return x_out, y_out

In [ ]:
import tensorflow as tf
import scipy

def weight_variable(shape):
  initial = tf.truncated_normal(shape, stddev=0.1)
  return tf.Variable(initial)

def bias_variable(shape):
  initial = tf.constant(0.1, shape=shape)
  return tf.Variable(initial)

def conv2d(x, W, stride):
  return tf.nn.conv2d(x, W, strides=[1, stride, stride, 1], padding='VALID')

x = tf.placeholder(tf.float32, shape=[None, 66, 200, 3])
y_ = tf.placeholder(tf.float32, shape=[None, 1])

x_image = x

#first convolutional layer
W_conv1 = weight_variable([5, 5, 3, 24])
b_conv1 = bias_variable([24])

h_conv1 = tf.nn.relu(conv2d(x_image, W_conv1, 2) + b_conv1)

#second convolutional layer
W_conv2 = weight_variable([5, 5, 24, 36])
b_conv2 = bias_variable([36])

h_conv2 = tf.nn.relu(conv2d(h_conv1, W_conv2, 2) + b_conv2)

#third convolutional layer
W_conv3 = weight_variable([5, 5, 36, 48])
b_conv3 = bias_variable([48])

h_conv3 = tf.nn.relu(conv2d(h_conv2, W_conv3, 2) + b_conv3)

#fourth convolutional layer
W_conv4 = weight_variable([3, 3, 48, 64])
b_conv4 = bias_variable([64])

h_conv4 = tf.nn.relu(conv2d(h_conv3, W_conv4, 1) + b_conv4)

#fifth convolutional layer
W_conv5 = weight_variable([3, 3, 64, 64])
b_conv5 = bias_variable([64])

h_conv5 = tf.nn.relu(conv2d(h_conv4, W_conv5, 1) + b_conv5)

#FCL 1
W_fc1 = weight_variable([1152, 1164])
b_fc1 = bias_variable([1164])

h_conv5_flat = tf.reshape(h_conv5, [-1, 1152])
h_fc1 = tf.nn.relu(tf.matmul(h_conv5_flat, W_fc1) + b_fc1)

keep_prob = tf.placeholder(tf.float32)
h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)

#FCL 2
W_fc2 = weight_variable([1164, 100])
b_fc2 = bias_variable([100])

h_fc2 = tf.nn.relu(tf.matmul(h_fc1_drop, W_fc2) + b_fc2)

h_fc2_drop = tf.nn.dropout(h_fc2, keep_prob)

#FCL 3
W_fc3 = weight_variable([100, 50])
b_fc3 = bias_variable([50])

h_fc3 = tf.nn.relu(tf.matmul(h_fc2_drop, W_fc3) + b_fc3)

h_fc3_drop = tf.nn.dropout(h_fc3, keep_prob)

#FCL 3
W_fc4 = weight_variable([50, 10])
b_fc4 = bias_variable([10])

h_fc4 = tf.nn.relu(tf.matmul(h_fc3_drop, W_fc4) + b_fc4)

h_fc4_drop = tf.nn.dropout(h_fc4, keep_prob)

#Output
W_fc5 = weight_variable([10, 1])
b_fc5 = bias_variable([1])

y = tf.identity(tf.matmul(h_fc4_drop, W_fc5) + b_fc5) #scale the atan output

In [ ]:
import os
import tensorflow as tf
from tensorflow.core.protobuf import saver_pb2
import driving_data
import model

LOGDIR = './save_identity_4_0.5'

sess = tf.InteractiveSession()

L2NormConst = 0.001

train_vars = tf.trainable_variables()

loss = tf.reduce_mean(tf.square(tf.subtract(model.y_, model.y))) + tf.add_n([tf.nn.l2_loss(v) for v in train_vars]) * L2NormConst
train_step = tf.train.AdamOptimizer(1e-4).minimize(loss)
sess.run(tf.initialize_all_variables())

# create a summary to monitor cost tensor
tf.summary.scalar("loss", loss)
# merge all summaries into a single op
merged_summary_op =  tf.summary.merge_all()

saver = tf.train.Saver(write_version = saver_pb2.SaverDef.V1)

# op to write logs to Tensorboard
logs_path = './save_identity_4_0.5'
summary_writer = tf.summary.FileWriter(logs_path, graph=tf.get_default_graph())

epochs = 30
batch_size = 100

# train over the dataset about 30 times
for epoch in range(epochs):
  for i in range(int(driving_data.num_images/batch_size)):
    xs, ys = driving_data.LoadTrainBatch(batch_size)
    train_step.run(feed_dict={model.x: xs, model.y_: ys, model.keep_prob: 0.5})
    if i % 10 == 0:
      xs, ys = driving_data.LoadValBatch(batch_size)
      loss_value = loss.eval(feed_dict={model.x:xs, model.y_: ys, model.keep_prob: 1.0})
      print("Epoch: %d, Step: %d, Loss: %g" % (epoch, epoch * batch_size + i, loss_value))

    # write logs at every iteration
    summary = merged_summary_op.eval(feed_dict={model.x:xs, model.y_: ys, model.keep_prob: 1.0})
    summary_writer.add_summary(summary, epoch * driving_data.num_images/batch_size + i)

    if i % batch_size == 0:
      if not os.path.exists(LOGDIR):
        os.makedirs(LOGDIR)
      checkpoint_path = os.path.join(LOGDIR, "model.ckpt")
      filename = saver.save(sess, checkpoint_path)
  print("Model saved in file: %s" % filename)

print("Run the command line:\n" \
          "--> tensorboard --logdir=./logs " \
          "\nThen open http://0.0.0.0:6006/ into your web browser")

In [1]:
#pip3 install opencv-python
import imageio
from skimage.transform import resize
import tensorflow as tf
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior() 
import scipy.misc
import model
import cv2
from subprocess import call
import math
from matplotlib.pyplot import imread
from PIL import Image 

sess = tf.InteractiveSession()
saver = tf.train.Saver()
saver.restore(sess, "save_identity_4_0.5/model.ckpt")

img = cv2.imread('steering_wheel_image.jpg',0)
rows,cols = img.shape

smoothed_angle = 0


#read data.txt
xs = []
ys = []
with open("driving_dataset/data.txt") as f:
    for line in f:
        xs.append("driving_dataset/" + line.split()[0])
        #the paper by Nvidia uses the inverse of the turning radius,
        #but steering wheel angle is proportional to the inverse of turning radius
        #so the steering wheel angle in radians is used as the output
        ys.append(float(line.split()[1]) * scipy.pi / 180)

#get number of images
num_images = len(xs)


i = math.ceil(num_images*0.8)
print("Starting frameofvideo:" +str(i))

while(cv2.waitKey(10) != ord('q')):
    full_image =imread("driving_dataset/" + str(i) + ".jpg")
    image = full_image[-150:]/255.0
    image=resize(image,[66,200])
    degrees = model.y.eval(feed_dict={model.x: [image], model.keep_prob: 1.0})[0][0] * 180.0 / scipy.pi
    #call("clear")
    #print("Predicted Steering angle: " + str(degrees))
    print("Steering angle: " + str(degrees) + " (pred)\t" + str(ys[i]*180/scipy.pi) + " (actual)")
    cv2.imshow("frame", cv2.cvtColor(full_image, cv2.COLOR_RGB2BGR))
    #make smooth angle transitions by turning the steering wheel based on the difference of the current angle
    #and the predicted angle
    smoothed_angle += 0.2 * pow(abs((degrees - smoothed_angle)), 2.0 / 3.0) * (degrees - smoothed_angle) / abs(degrees - smoothed_angle)
    M = cv2.getRotationMatrix2D((cols/2,rows/2),-smoothed_angle,1)
    dst = cv2.warpAffine(img,M,(cols,rows))
    cv2.imshow("steering wheel", dst)
    i += 1

cv2.destroyAllWindows()



Instructions for updating:
non-resource variables are not supported in the long term
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
INFO:tensorflow:Restoring parameters from save_identity_4_0.5/model.ckpt
Starting frameofvideo:36325
Steering angle: 5.401364395178035 (pred)	-10.79 (actual)
Steering angle: 5.950197842632063 (pred)	-10.08 (actual)
Steering angle: 6.092030989615609 (pred)	-9.380000000000003 (actual)
Steering angle: 7.740087334447986 (pred)	-7.559999999999999 (actual)
Steering angle: 7.924224111829652 (pred)	-5.949999999999999 (actual)
Steering angle: 8.660908678913364 (pred)	-3.23 (actual)
Steering angle: 6.5444217391022335 (pred)	-0.81 (actual)
Steering angle: 5.497418626136394 (pred)	1.21 (actual)
Steering angle: 4.911337168930525 (pred)	3.4300000000000006 (actual)
Steering angle: 2.7522067968100665 (pred)	4.44 (actual)
Steering angle: 2.5421614043592498 (pred)	5.340000000000001 (actual)
Steering angle: 

Steering angle: 6.5460182958207245 (pred)	46.69 (actual)
Steering angle: 5.927064845686877 (pred)	46.989999999999995 (actual)
Steering angle: 2.764414052404172 (pred)	47.09 (actual)
Steering angle: 1.97112023714916 (pred)	47.19 (actual)
Steering angle: 1.7660462619187602 (pred)	47.19 (actual)
Steering angle: 1.7165772294727677 (pred)	47.19 (actual)
Steering angle: 1.606372127214612 (pred)	47.19 (actual)
Steering angle: 1.544970861010313 (pred)	47.19 (actual)
Steering angle: 1.4795534435740465 (pred)	47.09 (actual)
Steering angle: 1.5342301750970564 (pred)	47.09 (actual)
Steering angle: 1.560349885700243 (pred)	47.39 (actual)
Steering angle: 2.2055613577679702 (pred)	47.7 (actual)
Steering angle: 2.6284194365005176 (pred)	0.0 (actual)
Steering angle: 3.0984826601343154 (pred)	47.7 (actual)
Steering angle: 4.277429019608934 (pred)	47.7 (actual)
Steering angle: 5.517272278506328 (pred)	47.7 (actual)
Steering angle: 5.895103402352625 (pred)	47.7 (actual)
Steering angle: 6.968555995992515 (

Steering angle: 9.189486773496895 (pred)	23.29 (actual)
Steering angle: 10.329662204478172 (pred)	22.89 (actual)
Steering angle: 12.517138715427828 (pred)	22.49 (actual)
Steering angle: 13.964014399578419 (pred)	22.29 (actual)
Steering angle: 13.99771284539597 (pred)	21.98 (actual)
Steering angle: 14.901082202758317 (pred)	21.68 (actual)
Steering angle: 14.066630307895045 (pred)	21.68 (actual)
Steering angle: 11.832328581316718 (pred)	21.68 (actual)
Steering angle: 9.10330003145509 (pred)	21.68 (actual)
Steering angle: 8.18724872035132 (pred)	21.68 (actual)
Steering angle: 7.824649345240975 (pred)	21.68 (actual)
Steering angle: 6.427207155208352 (pred)	21.68 (actual)
Steering angle: 3.8006425419543164 (pred)	21.68 (actual)
Steering angle: 2.698946475766188 (pred)	21.68 (actual)
Steering angle: 2.7485385583389466 (pred)	21.68 (actual)
Steering angle: 2.950294235571484 (pred)	21.68 (actual)
Steering angle: 2.7289542715544046 (pred)	22.08 (actual)
Steering angle: 2.840696166375813 (pred)	

Steering angle: 27.7098793872558 (pred)	15.030000000000001 (actual)
Steering angle: 27.3279095956593 (pred)	15.030000000000001 (actual)
Steering angle: 26.200545892013675 (pred)	15.030000000000001 (actual)
Steering angle: 25.898231474065156 (pred)	15.030000000000001 (actual)
Steering angle: 24.66207651731957 (pred)	15.030000000000001 (actual)
Steering angle: 24.718193351540393 (pred)	15.030000000000001 (actual)
Steering angle: 22.133789788485554 (pred)	15.030000000000001 (actual)
Steering angle: 18.997093714048898 (pred)	15.030000000000001 (actual)
Steering angle: 16.85472633369894 (pred)	15.030000000000001 (actual)
Steering angle: 16.6899446048778 (pred)	15.030000000000001 (actual)
Steering angle: 15.445347534318076 (pred)	15.030000000000001 (actual)
Steering angle: 11.222098136526622 (pred)	15.030000000000001 (actual)
Steering angle: 10.91305171337741 (pred)	15.030000000000001 (actual)
Steering angle: 10.373334434031234 (pred)	15.030000000000001 (actual)
Steering angle: 10.7147687604

Steering angle: 6.779630829115847 (pred)	-0.5 (actual)
Steering angle: 6.086754668481774 (pred)	-0.5 (actual)
Steering angle: 5.741136851195374 (pred)	-0.5 (actual)
Steering angle: 4.937022095304358 (pred)	-0.5 (actual)
Steering angle: 5.667423742125304 (pred)	-0.3 (actual)
Steering angle: 6.548670970539628 (pred)	-0.3 (actual)
Steering angle: 6.489693567717037 (pred)	-0.3 (actual)
Steering angle: 4.844149451838756 (pred)	-0.3 (actual)
Steering angle: 3.2650370364884878 (pred)	-0.3 (actual)
Steering angle: 3.424016306166293 (pred)	-0.3 (actual)
Steering angle: 0.39006431281490617 (pred)	-0.3 (actual)
Steering angle: 0.12109157002150982 (pred)	-0.4 (actual)
Steering angle: 0.15232698559133015 (pred)	-0.5 (actual)
Steering angle: 0.6235184710986494 (pred)	-0.5 (actual)
Steering angle: 0.8012458629961993 (pred)	-0.61 (actual)
Steering angle: 3.1433473980277786 (pred)	-0.71 (actual)
Steering angle: 3.4138239563773416 (pred)	-0.91 (actual)
Steering angle: 2.12583917633217 (pred)	-1.11 (actu

...

Steering angle: 1.6332869145258704 (pred)	15.63 (actual)
Steering angle: 1.6329377211045535 (pred)	15.63 (actual)
Steering angle: 1.646501836465964 (pred)	15.33 (actual)
Steering angle: 1.6908086332816021 (pred)	14.52 (actual)
Steering angle: 1.6145269461967842 (pred)	13.41 (actual)
Steering angle: 1.6525956458660849 (pred)	12.709999999999999 (actual)
Steering angle: 1.6626756176988484 (pred)	11.8 (actual)
Steering angle: 1.7093463002975455 (pred)	11.290000000000001 (actual)
Steering angle: 1.6194711493822271 (pred)	10.69 (actual)
Steering angle: 1.6956134579192945 (pred)	9.78 (actual)
Steering angle: 1.7271127962675124 (pred)	8.87 (actual)
Steering angle: 1.7900489340443602 (pred)	8.27 (actual)
Steering angle: 1.786456894871168 (pred)	7.87 (actual)
Steering angle: 1.9118000442075993 (pred)	7.460000000000001 (actual)
Steering angle: 1.9680598855141713 (pred)	7.06 (actual)
Steering angle: 1.8351375740765532 (pred)	6.45 (actual)
Steering angle: 1.9208933738704135 (pred)	6.150000000000001

Steering angle: 1.7705026401870763 (pred)	-3.63 (actual)
Steering angle: 1.5299587455448596 (pred)	-3.63 (actual)
Steering angle: 1.4318454259951536 (pred)	-3.63 (actual)
Steering angle: 1.3761536643284564 (pred)	-3.7300000000000004 (actual)
Steering angle: 1.3532437157483528 (pred)	-3.7300000000000004 (actual)
Steering angle: 1.4177220888954483 (pred)	-3.7300000000000004 (actual)
Steering angle: 1.5935432176843298 (pred)	-3.7300000000000004 (actual)
Steering angle: 1.4888475167654356 (pred)	-3.7300000000000004 (actual)
Steering angle: 1.3131646993072468 (pred)	-3.7300000000000004 (actual)
Steering angle: 1.2879212806302318 (pred)	-3.7300000000000004 (actual)
Steering angle: 1.117766113922993 (pred)	-3.7300000000000004 (actual)
Steering angle: 0.990003796959908 (pred)	-3.829999999999999 (actual)
Steering angle: 0.8908130150686325 (pred)	-4.13 (actual)
Steering angle: 1.2943914974866437 (pred)	-4.24 (actual)
Steering angle: 1.3562383268125802 (pred)	-4.24 (actual)
Steering angle: 1.1138

Steering angle: 1.435741088421294 (pred)	-2.62 (actual)
Steering angle: 1.359257697312549 (pred)	-2.62 (actual)
Steering angle: 1.3858951149104812 (pred)	-2.52 (actual)
Steering angle: 1.3334821642105075 (pred)	-2.32 (actual)
Steering angle: 1.4050707642835865 (pred)	-2.12 (actual)
Steering angle: 1.4344940452892376 (pred)	-1.92 (actual)
Steering angle: 1.4618427630562034 (pred)	-1.6100000000000003 (actual)
Steering angle: 1.347601125720276 (pred)	-1.21 (actual)
Steering angle: 1.334137222040594 (pred)	-0.81 (actual)
Steering angle: 1.3588013552986276 (pred)	-0.2 (actual)
Steering angle: 1.4070535468552998 (pred)	0.5 (actual)
Steering angle: 1.305448719979248 (pred)	1.71 (actual)
Steering angle: 1.2376678436467456 (pred)	2.42 (actual)
Steering angle: 1.2406261693310072 (pred)	2.92 (actual)
Steering angle: 1.2734697742852483 (pred)	3.23 (actual)
Steering angle: 1.3093468370046213 (pred)	3.4300000000000006 (actual)
Steering angle: 1.2536884792318337 (pred)	3.7300000000000004 (actual)
Ste

Steering angle: -0.6429190898275418 (pred)	-9.28 (actual)
Steering angle: -0.6399031344221581 (pred)	-9.380000000000003 (actual)
Steering angle: -0.8112004368246697 (pred)	-9.58 (actual)
Steering angle: -1.105416171408258 (pred)	-9.58 (actual)
Steering angle: -0.9097026502702433 (pred)	-9.78 (actual)
Steering angle: -1.323578389913989 (pred)	-9.88 (actual)
Steering angle: -1.526068739369126 (pred)	-10.08 (actual)
Steering angle: -1.3124914987871774 (pred)	-10.08 (actual)


KeyboardInterrupt: 

In [9]:
import prettytable

table=prettytable.PrettyTable()

table.field_names=['Model activation','keepprob','Loss']

table.add_row(['identity','1e-4','0.4013'])
#table.add_row(['identity','1e-3','0.4425',""])



print(table)

+------------------+----------+--------+
| Model activation | keepprob |  Loss  |
+------------------+----------+--------+
|     identity     |   1e-4   | 0.4013 |
+------------------+----------+--------+


1. This model comparatively doing good when we compared below model but sometimes it will do mistakes but it is not best compare to base model.

In [11]:
import prettytable

table=prettytable.PrettyTable()

table.field_names=['Model activation','keepprob','Loss']

#table.add_row(['identity','1e-4','0.4013'])
table.add_row(['identity','1e-3','0.4425'])



print(table)

+------------------+----------+--------+
| Model activation | keepprob |  Loss  |
+------------------+----------+--------+
|     identity     |   1e-3   | 0.4425 |
+------------------+----------+--------+


1. "Above model is not performing well compared to base model, in some situation model supposed to predict the -VE steering angle but it is predicting the +VE steering angle."